### RoadPercption Dataset Example

Please make sure you have the data (imgs, gt folders) located in this directory to match the paths in data.csv file - link provided in the README file

In [2]:
import os
from os.path import isfile, basename
import cv2
import torch
import blosc
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


class RoadPerceptionDataset():
    def __init__(self, pth):
        self.gt_h = 224
        self.gt_w = 280

        self.df = pd.read_csv(pth, names=["img","gt"])

        # layer index + BGR color values for visulization
        self.layers_colormap = {
                    'road'            : (0, [0,0,255]),     # red
                    'stop_lines'      : (1, [0,255,255]),   # yellow
                    'speed_bumps'     : (2, [255,0,255]),   # purple
                    'dashed_line'     : (3, [255,255,0]),   # cyan
                    'cont_line'       : (4, [255,255,255]), # white
                    'zebra_crosswalk' : (5, [127,255,0]),   # light green
                    'parking'         : (6, [33,140,33]),   # dark green
                    'border_parking'  : (7, [180,180,180]), # gray
                    'island'          : (8, [0,153,255]),   # orange
                    'empty_crosswalk' : (9, [255,204,102])  # light blue
        }

        # order in which the layers are stacked for GT visualization, first on top
        self.layers_vis_order = [
                    'cont_line',
                    'dashed_line',
                    'speed_bumps',
                    'stop_lines',
                    'island',
                    'zebra_crosswalk',
                    'empty_crosswalk',
                    'road',
                    'border_parking',
                    'parking'
        ]
        self.layers_vis_order = [list(self.layers_colormap.keys()).index(c) for c in self.layers_vis_order]

        # BGR color values corresponding to each layer (first is background - black)
        self.palettes = torch.cat((torch.ByteTensor([[0,0,0]]), torch.ByteTensor([v[1] for v in self.layers_colormap.values()])))

        # 90-degree FOV mask for the top-down visualization
        self.fov_mask = np.zeros((self.gt_h, self.gt_w))
        xx, yy = np.meshgrid(np.arange(0,self.gt_w), np.arange(0,self.gt_h))
        hh = (self.gt_h-1)/2
        self.fov_mask[yy+xx < hh] = 1
        self.fov_mask[yy-xx > hh] = 1
    
    # loads path to a binary GT file, returns it as a (224,280,11) numpy array
    def load_gt(self, pth):
        with open(pth, 'rb') as f:
            gt = np.frombuffer(blosc.decompress(f.read()), dtype=np.bool_).reshape(self.gt_h,self.gt_w,-1)
        return gt
    
    # gets an (h,w,c) array of layers, returns a (h,w,3) RGB visulization of it
    def layers_to_rgb(self, layers, show_masks=True, show_car=True, alpha=60):
        rgb = torch.zeros(layers.shape[:2], dtype=torch.long)
        for i in self.layers_vis_order[::-1]:
            rgb[layers[...,i]]=i+1
        rgb = self.palettes[rgb]
        if show_masks:
            occ_fov_mask = np.logical_or(self.fov_mask, layers[...,-1])
            rgb[occ_fov_mask] = (rgb[occ_fov_mask].to(torch.int32)*alpha//100 + 255*(100-alpha)//100).to(torch.uint8)
        if show_car:
            hh = rgb.shape[0]//2
            rgb[hh-5:hh+6,:10] = 255
        return rgb
    
    # returns a single pair of img-gt paths from the dataset, either randomly or by a specified index
    def get_sample_paths(self, idx=None):
        return self.df.iloc[idx if idx is not None else np.random.randint(len(self.df))]
    
    # gets a pair of img-gt paths, returns a side-by-side visualization of them
    def get_sample(self, img_pth, gt_pth):
        gt = self.load_gt(gt_pth)
        gt_vis = self.layers_to_rgb(gt)
        img = cv2.resize(cv2.imread(img_pth), gt_vis.shape[:2][::-1])
        return np.concatenate((img,gt_vis), axis=1)
    
    # gets a pair of img-gt paths, presents their side-by-side visualization
    def visualize_sample(self, img_pth, gt_pth):
        img_gt_sample = self.get_sample(img_pth, gt_pth)
        trip, img_ts, cam = gt_pth.rsplit('/',3)[1], basename(img_pth), gt_pth.rsplit('/',2)[1].split('_')[1]
        plt.figure(figsize=(12,30))
        plt.title(f"trip: {trip}, img: {img_ts}, cam: {cam}")
        plt.imshow(img_gt_sample[...,::-1])

### Load Dataset

Pairs of image vs GT of binary top-down layers, which cover 75x60 meters (more details below)

In [3]:
datafile = "data.csv"

rp_data = RoadPerceptionDataset(datafile)

### Visualize Samples

For visualization, GT layers are stacked on top of each other with the following colors:

- Road (red)
- Stop lines (yellow)
- Speed bumps (purple)
- Dashed lane dividers (cyan)
- Continuous lane dividers (white)
- Marked crosswalks (light green)
- Parking spaces (dark green)
- Parking borders (gray)
- Painted islands (orange)
- Unmarked crosswalks (light blue)
- Occlusion mask - buildings, walls and other static obstacles (transparent)

Run this cell to visualize one sample at a time.

In [ ]:
img_pth, gt_pth = rp_data.get_sample_paths()

print(f"selected sample:\n{img_pth}\n{gt_pth}\n")

if isfile(img_pth) and isfile(gt_pth):
    rp_data.visualize_sample(img_pth, gt_pth)
else:
    print("image and/or gt file not found")